In [18]:
vdw_type = "protor"

rs = 1.4
η = 0.3665
#σ_t = 0.5
#σ_r = 0.25
σ_t = 0.25
σ_r = 0.125
overlap_jump = 0.0
overlap_slope = 0.85
delaunay_eps = 10.0
bnds = 130.0
comment = ""

energy_specifier = "oj_$(replace(string(overlap_jump), "." => "_"))_os_$(replace(string(overlap_slope), "." => "_"))"
simulation_time_minutes = 24.0 * 60.0

1440.0

In [19]:
T = 2.5

input_specifier = "5_1stm"
template_folder = "assets/input/jld2/$(input_specifier)_init/"

simulations_per_combination = 1

open("configs/$(input_specifier)_config.txt", "w") do io
    println(io,"ArrayTaskID input_string")
    i = 0
    for file in readdir("../$(template_folder)")
        i+=1
        template_file = "$(template_folder)$(file)"
        output_directory = "assets/output/assembly/$(input_specifier)_$(energy_specifier)/"
        mol_type = split(file, ".")[1]
        for j in 1:simulations_per_combination
            name = "$(mol_type)_$(vdw_type)_$(energy_specifier)"
            input_string = escape_string("name=\"$name\";T=$(T);rs=$rs;η=$η;σ_t=$σ_t;σ_r=$σ_r;overlap_jump=$overlap_jump;overlap_slope=$overlap_slope;bnds=$bnds;simulation_time_minutes=$simulation_time_minutes;output_directory=\"$output_directory\";template_file=\"$template_file\";delaunay_eps=$delaunay_eps;comment=\"$comment\"")
            println(io, "$((i-1) * simulations_per_combination + j) $input_string")
        end
    end
end


In [20]:
total_simulations = length(readlines("configs/$(input_specifier)_config.txt")) - 1

50

In [22]:
hours = Int(round(simulation_time_minutes / 60.0))
days = hours ÷ 24
remaining_hours = hours % 24
remaining_hours_string = remaining_hours < 10 ? "0$(remaining_hours)" : string(remaining_hours)
buffer_time_string = simulation_time_minutes < 5 ? "0$(Int(simulation_time_minutes)+2)" : "30"

open("$(input_specifier)_script.job", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#SBATCH --job-name=SolvationSimulations")
    println(io, "#SBATCH --time=0$(days)-$(remaining_hours_string):$(buffer_time_string)")
    println(io, "#SBATCH --ntasks=1")
    println(io, "#SBATCH --cpus-per-task=1")
    println(io, "#SBATCH --array=1-$(total_simulations)")
    println(io, "#SBATCH --chdir=/work/spirandelli/MorphoMolMonteCarlo/")
    println(io, "#SBATCH -o ./hpc/out/%a.out # STDOUT")
    println(io, "")
    println(io, "export http_proxy=http://proxy2.uni-potsdam.de:3128 #Setting proxy, due to lack of Internet on compute nodes.")
    println(io, "export https_proxy=http://proxy2.uni-potsdam.de:3128")
    println(io, "")
    println(io, "module purge")
    println(io, "module load lang/Julia/1.7.3-linux-x86_64")
    println(io, "")
    println(io, "# Specify the path to the config file")
    println(io, "config=hpc/configs/$(input_specifier)_config.txt")
    println(io, "")
    println(io, "# Extract the variables from config file")
    println(io, "config_string=\$(awk -v ArrayTaskID=\$SLURM_ARRAY_TASK_ID '\$1==ArrayTaskID {print \$2}' \$config)")
    println(io, "")
    #println(io, "julia --project -e \"$(escape_string("push!(LOAD_PATH, \"@pkglock\"); using PkgLock; PkgLock.instantiate_precompile()"))\"")
    println(io, "julia -e \"$(escape_string("include(\"hpc/rwm_call.jl\"); rwm_call(\"\$config_string\")"))\"")
    println(io, "")
    println(io, "# copy back results")
    println(io, "mkdir -p ~/outputs/ && cp -r assets/output/assembly/* ~/outputs/")
end